# Plan and Execute Pattern using Semantic Kernel

This notebook demonstrates the implementation of the Plan and Execute pattern using Semantic Kernel. This pattern improves agent performance by:

1. Breaking down complex tasks into manageable sub-tasks (Planning)
2. Executing each sub-task in sequence
3. Adapting to feedback during execution

## Architecture Overview

The Plan and Execute pattern involves:
- **Planner**: Responsible for generating a structured plan of sub-tasks
- **Executor**: Handles the execution of each sub-task
- **Memory**: Maintains context between steps
- **Tools**: Custom functions that can be called during execution

![Plan and Execute Pattern](../1_agentic-design-ptn/images/planning.png)

In [ ]:
# Setup and Dependencies
import os
import json
import requests
import asyncio
import httpx
from urllib.parse import urljoin
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.planning import SequentialPlanner
from semantic_kernel.planning.sequential_planner.sequential_planner_config import SequentialPlannerConfig
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from typing import List, Dict, Any
import nest_asyncio
from IPython.display import display, Markdown
import logging

# Apply nest_asyncio to allow nested event loops (required for Jupyter)
nest_asyncio.apply()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Helper function to display markdown content
def display_md(text: str):
    display(Markdown(text))

In [ ]:
# Load environment variables and configure Azure OpenAI
from dotenv import load_dotenv
load_dotenv()

# Get API keys from environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_KEY")
DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

# Check that environment variables are set
if not AZURE_OPENAI_ENDPOINT or not AZURE_OPENAI_API_KEY or not DEPLOYMENT_NAME:
    raise ValueError("Please set AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_KEY, and AZURE_OPENAI_DEPLOYMENT_NAME in .env file")
    
print(f"Azure OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Deployment Name: {DEPLOYMENT_NAME}")

# Check if Google Search API keys are available
if not GOOGLE_API_KEY or not GOOGLE_CSE_ID:
    print("Warning: GOOGLE_API_KEY or GOOGLE_CSE_ID not set. Web search functionality will be limited to demo mode.")

In [ ]:
# Initialize Semantic Kernel with Azure OpenAI
kernel = sk.Kernel()

# Add Azure OpenAI service
kernel.add_service(
    AzureChatCompletion(
        service_id="azure_chat_completion",
        deployment_name=DEPLOYMENT_NAME,
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY
    )
)

print("Semantic Kernel initialized with Azure OpenAI service")

## Creating Native Functions (Tools)

First, we'll create several native functions that our planner can use during execution. These functions represent the tools that our agent can use to complete tasks.

In [ ]:
# Create a plugin with native functions that can be used by the planner
@kernel_function(
    description="Searches for information about a product.",
    name="search_product"
)
def search_product(query: str) -> str:
    """
    Searches for information about a specified product using web search.
    
    Args:
        query: The product to search for.
        
    Returns:
        Information about the product.
    """
    logger.info(f"Searching for product: {query}")
    
    # Simulated product database as fallback
    static_product_database = {
        "smartphone": "Latest model with 6.7-inch display, 128GB storage, 12MP camera, and 5G capability.",
        "laptop": "Premium laptop with 16GB RAM, 512GB SSD, Intel i7 processor, and 15-inch display.",
        "headphones": "Wireless noise-cancelling headphones with 30-hour battery life and premium sound quality.",
        "smartwatch": "Fitness tracking smartwatch with heart rate monitor, GPS, and 7-day battery life.",
        "tablet": "10-inch tablet with 64GB storage, 8MP camera, and all-day battery life."
    }
    
    # First try web search
    results = google_search(query, num=3)
    
    if results:
        # Process search results
        product_info = f"Found information about {query} from web search:\n\n"
        
        for i, result in enumerate(results):
            title = result.get("title", "Untitled")
            link = result.get("link", "")
            snippet = result.get("snippet", result.get("text_content", ""))
            
            product_info += f"{i+1}. {title}\n"
            product_info += f"   URL: {link}\n"
            if snippet:
                product_info += f"   Summary: {snippet}\n"
            product_info += "\n"
            
        return product_info
    
    # Fallback to static database if web search fails
    for product_name, product_info in static_product_database.items():
        if product_name.lower() in query.lower():
            return f"Found information about {product_name} (from local database): {product_info}"
    
    return f"Could not find specific information about '{query}'. Available products in our local database are: {', '.join(static_product_database.keys())}."

@kernel_function(
    description="Compares features between two products.",
    name="compare_products"
)
def compare_products(product1: str, product2: str) -> str:
    """
    Compares features between two products using web search.
    
    Args:
        product1: First product to compare.
        product2: Second product to compare.
        
    Returns:
        Comparison between the products.
    """
    logger.info(f"Comparing products: {product1} vs {product2}")
    
    # Static comparisons as fallback
    static_comparisons = {
        ("smartphone", "tablet"): "Smartphones are more portable with calling capabilities, while tablets offer larger screens better for media consumption and productivity.",
        ("laptop", "tablet"): "Laptops have more processing power and a physical keyboard, while tablets are more portable with touchscreen interfaces.",
        ("headphones", "smartwatch"): "Headphones are dedicated to audio quality, while smartwatches offer fitness tracking and notifications with limited audio capabilities."
    }
    
    # Search for each product
    product1_results = google_search(product1, num=2)
    product2_results = google_search(product2, num=2)
    
    if product1_results and product2_results:
        comparison = f"Comparison between {product1} and {product2} based on web search:\n\n"
        
        # First product info
        comparison += f"## {product1.capitalize()} Information:\n"
        for i, result in enumerate(product1_results[:2]):
            title = result.get("title", "Untitled")
            snippet = result.get("snippet", result.get("text_content", "No description available"))
            comparison += f"{i+1}. {title}\n   {snippet}\n\n"
        
        # Second product info
        comparison += f"## {product2.capitalize()} Information:\n"
        for i, result in enumerate(product2_results[:2]):
            title = result.get("title", "Untitled")
            snippet = result.get("snippet", result.get("text_content", "No description available"))
            comparison += f"{i+1}. {title}\n   {snippet}\n\n"
        
        # Add comparison prompt
        comparison += f"Based on the information above, here are the key differences between {product1} and {product2}:"
        
        return comparison
    
    # Fallback to static comparisons
    for (p1, p2), comparison in static_comparisons.items():
        if (p1 in product1.lower() and p2 in product2.lower()) or (p1 in product2.lower() and p2 in product1.lower()):
            return f"Comparison (from local database): {comparison}"
    
    return f"No comparison data available between '{product1}' and '{product2}'. Try with more specific product types."

@kernel_function(
    description="Provides recommendations based on user preferences.",
    name="recommend_product"
)
def recommend_product(preferences: str) -> str:
    """
    Recommends products based on user preferences using web search.
    
    Args:
        preferences: User preferences for product recommendations.
        
    Returns:
        Product recommendations.
    """
    logger.info(f"Generating recommendations based on preferences: {preferences}")
    
    # Create a search query based on user preferences
    search_query = f"best products for {preferences}"
    
    # Perform web search
    results = google_search(search_query, num=3)
    
    if results:
        recommendations = f"Based on your preferences for '{preferences}', here are some recommended products:\n\n"
        
        for i, result in enumerate(results):
            title = result.get("title", "Untitled")
            snippet = result.get("snippet", result.get("text_content", "No description available"))
            
            recommendations += f"{i+1}. {title}\n"
            recommendations += f"   {snippet}\n\n"
            
        return recommendations
    
    # Fallback to static recommendations if web search fails
    preferences = preferences.lower()
    
    if "portable" in preferences or "mobile" in preferences:
        if "powerful" in preferences or "performance" in preferences:
            return "Based on your preference for portability and performance (from local database), I recommend a high-end ultrabook laptop or a flagship smartphone."
        else:
            return "For maximum portability (from local database), I recommend a lightweight tablet or a compact smartphone."
    
    if "audio" in preferences or "music" in preferences or "sound" in preferences:
        return "For audio enthusiasts (from local database), I recommend our premium noise-cancelling headphones with exceptional sound quality."
    
    if "fitness" in preferences or "health" in preferences or "exercise" in preferences:
        return "To track your fitness activities (from local database), I recommend our smartwatch with heart rate monitoring and exercise tracking."
    
    return "Based on your preferences (from local database), I might need more specific information to make a tailored recommendation."

# Register functions with the kernel
tool_functions_plugin = kernel.add_functions_from_object(search_product, "ProductTools")
kernel.add_functions_from_object(compare_products, "ProductTools")
kernel.add_functions_from_object(recommend_product, "ProductTools")

print("Native functions registered as tools")

## Creating Semantic Functions

Next, we'll create semantic functions that use natural language to perform tasks. These will be combined with our native functions in the planner.

In [ ]:
# Create a semantic function for summarizing product information
summarize_prompt = """
You are a helpful product specialist.
Summarize the following product information in a concise and helpful manner.
Focus on the key features and benefits that would be most relevant to customers.

Product Information:
{{$input}}

Summary:
"""

summarize_function = kernel.create_function_from_prompt(
    function_name="summarize_product_info",
    plugin_name="ProductTools",
    prompt=summarize_prompt,
    description="Summarizes product information in a concise and helpful manner."
)

# Create a semantic function for generating product comparisons
compare_prompt = """
You are a helpful product specialist.
Create a detailed comparison between the products based on the information provided.
Include pros and cons for each product and mention which types of users would prefer each option.

Product Information:
{{$input}}

Comparison:
"""

compare_function = kernel.create_function_from_prompt(
    function_name="generate_detailed_comparison",
    plugin_name="ProductTools",
    prompt=compare_prompt,
    description="Generates a detailed comparison between products."
)

print("Semantic functions created")

## Setting Up the Planner

Now we'll configure the sequential planner, which will break down complex tasks into a sequence of steps using our available functions.

In [ ]:
# Configure the sequential planner
planner_config = SequentialPlannerConfig(
    relevancy_threshold=0.7,
    max_tokens=1000
)

# Create the planner
planner = SequentialPlanner(kernel, planner_config)

print("Sequential planner configured")

## Executing a Plan

Let's put our plan and execute pattern to work with a sample scenario. We'll ask the agent to help a customer find the right product based on their needs.

In [ ]:
async def execute_plan_and_display(goal: str):
    """Execute a plan based on the specified goal and display the results."""
    print(f"🎯 Goal: {goal}")
    print("\n⚙️ Generating plan...\n")
    
    # Create and display the plan
    plan = await planner.create_plan(goal)
    
    plan_steps = "\n".join([f"Step {i+1}: {step.description}" for i, step in enumerate(plan.steps)])
    display_md(f"### Generated Plan\n{plan_steps}")
    
    print("\n🚀 Executing plan...\n")
    
    # Execute the plan
    result = await kernel.run(plan)
    
    print("\n✅ Plan execution completed\n")
    display_md(f"### Result\n{result}")
    
    return plan, result

# Example goal
goal = "I need a recommendation for a portable device with good battery life for a college student."

# Execute the plan
await execute_plan_and_display(goal)

## Try Different Goals

Let's try different types of user requests to see how our planner adapts.

In [ ]:
# Let's try a comparison request
comparison_goal = "Compare the features of a laptop and a tablet for someone who needs to do both schoolwork and watch movies."

await execute_plan_and_display(comparison_goal)

In [ ]:
# Let's try a specific product search request
search_goal = "I want to know the camera quality and storage options for the smartphone."

await execute_plan_and_display(search_goal)

## Understanding the Plan and Execute Pattern

The Plan and Execute pattern we've demonstrated offers several advantages:

1. **Task Decomposition**: Complex tasks are broken down into simpler, manageable steps
2. **Tool Selection**: The planner automatically selects the appropriate tools for each step
3. **Adaptability**: If a step fails, the planner can adapt by trying alternative approaches
4. **Explainability**: The plan provides transparency into how the agent approaches problems

This pattern is particularly useful for tasks that require multiple steps or the use of various tools to complete.

## Practical Applications

The Plan and Execute pattern can be applied to various real-world scenarios:

- **Customer Support**: Helping customers troubleshoot complex issues step by step
- **Research Assistance**: Breaking down research questions into specific search queries and synthesis steps
- **Task Automation**: Creating workflows that combine multiple API calls and data transformations
- **Product Recommendations**: Gathering user preferences and matching them to suitable products

## Next Steps

To extend this pattern, consider:

1. Adding error handling to retry failed steps
2. Implementing dynamic replanning based on execution results
3. Incorporating user feedback between steps
4. Adding more specialized tools for specific domains

In [ ]:
# Web search functions from llm_websearch_crawling.ipynb
def google_search(query, num=5, search_type="web"):
    """
    Perform a web search using Google Custom Search API.
    
    Args:
        query: The search query
        num: Number of results to return
        search_type: Type of search (web or image)
        
    Returns:
        List of search results
    """
    # Check if Google API keys are available
    if not GOOGLE_API_KEY or not GOOGLE_CSE_ID:
        logger.warning("Google API keys not set. Using demo mode with simulated results.")
        return _get_demo_search_results(query)
    
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num,
    }
    
    if search_type == "image":
        params["searchType"] = "image"
        
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for non-200 status codes
        results = response.json()
        return results.get("items", [])
    except Exception as e:
        logger.error(f"Error performing Google search: {e}")
        return _get_demo_search_results(query)

def _get_demo_search_results(query):
    """
    Generate simulated search results when API keys are not available.
    
    Args:
        query: The search query
        
    Returns:
        List of simulated search results
    """
    # Demo product database with sample search results
    demo_database = {
        "smartphone": [
            {"title": "Latest Smartphone Models 2025", "link": "https://example.com/smartphones", 
             "snippet": "The latest smartphone models feature 6.7-inch displays, 128GB storage, 12MP cameras, and 5G capability."},
            {"title": "Comparing Top Smartphones", "link": "https://example.com/smartphone-comparison",
             "snippet": "Compare the best smartphones with features like high-resolution cameras, long battery life, and fast processors."}
        ],
        "laptop": [
            {"title": "Premium Laptops Guide 2025", "link": "https://example.com/laptops", 
             "snippet": "Premium laptops with 16GB RAM, 512GB SSD, Intel i7 processors, and 15-inch displays for productivity."},
            {"title": "Best Laptops for Students", "link": "https://example.com/student-laptops",
             "snippet": "Find the best laptops for students with good battery life, lightweight design, and powerful performance."}
        ],
        "headphones": [
            {"title": "Wireless Headphones Review", "link": "https://example.com/headphones", 
             "snippet": "Wireless noise-cancelling headphones with 30-hour battery life and premium sound quality for music lovers."},
            {"title": "Top Audio Devices of 2025", "link": "https://example.com/audio-devices",
             "snippet": "Discover the best headphones with features like active noise cancellation, spatial audio, and comfortable design."}
        ],
        "smartwatch": [
            {"title": "Fitness Tracking Smartwatches", "link": "https://example.com/smartwatches", 
             "snippet": "Fitness tracking smartwatches with heart rate monitoring, GPS, and 7-day battery life for active lifestyles."},
            {"title": "Health Monitoring Wearables", "link": "https://example.com/health-wearables",
             "snippet": "Smartwatches that help you monitor your health with features like ECG, sleep tracking, and workout detection."}
        ],
        "tablet": [
            {"title": "Tablet Buyer's Guide 2025", "link": "https://example.com/tablets", 
             "snippet": "10-inch tablets with 64GB storage, 8MP cameras, and all-day battery life for work and entertainment."},
            {"title": "Best Tablets for Artists", "link": "https://example.com/artist-tablets",
             "snippet": "Tablets with pressure-sensitive displays, high resolution, and precise stylus support for digital art creation."}
        ]
    }
    
    # Find matching products
    results = []
    for product, product_results in demo_database.items():
        if product.lower() in query.lower():
            results.extend(product_results)
    
    # If no specific product matches, return general tech products
    if not results:
        for product_results in demo_database.values():
            results.append(product_results[0])
            if len(results) >= 3:
                break
    
    return results[:5]  # Return at most 5 results

async def extract_text_from_url(url):
    """
    Extract text content from a webpage URL.
    
    Args:
        url: The URL to extract text from
        
    Returns:
        Extracted text content
    """
    # For demo purposes, we'll generate some simulated content
    # In a real implementation, you would use httpx to fetch and parse the content
    
    # Simple simulation of webpage content based on URL
    if "example.com" in url:
        product_type = ""
        for product in ["smartphone", "laptop", "headphones", "smartwatch", "tablet"]:
            if product in url:
                product_type = product
                break
        
        if product_type:
            return f"This is a webpage about {product_type}s. It contains detailed information about features, specifications, and user reviews."
        else:
            return "This is a generic technology webpage with product information and reviews."
    
    return "Could not extract content from the provided URL."

async def add_context_async(urls):
    """
    Fetch and extract content from multiple URLs asynchronously.
    
    Args:
        urls: List of URLs to fetch content from
        
    Returns:
        List of extracted text content
    """
    tasks = [extract_text_from_url(url) for url in urls]
    results = await asyncio.gather(*tasks)
    return results